In [17]:
from pysal.model import spreg
import pandas as pd
import geopandas as gpd
import seaborn as sns
import osmnx as ox

from libpysal.weights import lat2W
from libpysal import weights
sns.set(style="whitegrid")

In [18]:
data = pd.read_csv("!Dep_fixed1.csv")
data.columns

data = data.dropna()

In [19]:
X = data.iloc[:,6:12]
y = data.iloc[:,5]

#X = X.fillna(X.mean())
#y = y.fillna(y.mean())

X['not deprived'] = X['not deprived']/X['All households']*100
X['deprived in one'] = X['deprived in one']/X['All households']*100
X['deprived in two'] = X['deprived in two']/X['All households']*100
X['deprived in three'] = X['deprived in three']/X['All households']*100
X['deprived in four'] = X['deprived in four']/X['All households']*100

#X = X.iloc[:,1:]
#print(X)
#print(y)

In [20]:
variable_names = [
    #"not deprived",
    "deprived in one",
    #"deprived in two",
    #"deprived in three",
    "deprived in four"
    ]

m1 = spreg.OLS(y.values, X[variable_names].values, name_x=variable_names,)
print(m1.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: ORDINARY LEAST SQUARES
-----------------------------------------
Data set            :     unknown
Weights matrix      :        None
Dependent Variable  :     dep_var                Number of Observations:         323
Mean dependent var  :     37.3657                Number of Variables   :           3
S.D. dependent var  :      6.1969                Degrees of Freedom    :         320
R-squared           :      0.3842
Adjusted R-squared  :      0.3804
Sum squared residual:    7614.447                F-statistic           :     99.8243
Sigma-square        :      23.795                Prob(F-statistic)     :   2.044e-34
S.E. of regression  :       4.878                Log likelihood        :    -968.681
Sigma-square ML     :      23.574                Akaike info criterion :    1943.363
S.E of regression ML:      4.8553                Schwarz criterion     :    1954.696

-----------------------------------------------------------------------------

In [21]:
import pyproj
bng=pyproj.Proj(init='epsg:27700')
wgs84 = pyproj.Proj(init='epsg:4326')

for i in  range(len(data)):
    data.iloc[i,12] = pyproj.transform(bng, wgs84, data.iloc[i,12], data.iloc[i,13])[0]
    data.iloc[i,13] = pyproj.transform(bng, wgs84, data.iloc[i,12], data.iloc[i,13])[1]

from libpysal.weights import KNN
data["geometry"] = gpd.points_from_xy(data['x'], data['y'])

w = KNN.from_dataframe(data, k=5)

c:\Users\1\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
c:\Users\1\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
C:\Users\1\AppData\Local\Temp\ipykernel_19248\2512082250.py:6: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  data.iloc[i,12] = pyproj.

In [22]:
#gg = lat2W(173,2)
w = KNN.from_dataframe(data, k=8)

m3 = spreg.GM_Lag(y.values, X[variable_names].values, w=w, name_x = variable_names)
print(m3.summary)

REGRESSION
----------
SUMMARY OF OUTPUT: SPATIAL TWO STAGE LEAST SQUARES
--------------------------------------------------
Data set            :     unknown
Weights matrix      :     unknown
Dependent Variable  :     dep_var                Number of Observations:         323
Mean dependent var  :     37.3657                Number of Variables   :           4
S.D. dependent var  :      6.1969                Degrees of Freedom    :         319
Pseudo R-squared    :      0.4183
Spatial Pseudo R-squared:  0.3853

------------------------------------------------------------------------------------
            Variable     Coefficient       Std.Error     z-Statistic     Probability
------------------------------------------------------------------------------------
            CONSTANT      -4.3027208       5.0148981      -0.8579877       0.3908993
     deprived in one       1.0064934       0.1514407       6.6461217       0.0000000
    deprived in four       8.8694208       1.5557321       

In [23]:
from sklearn.metrics import mean_squared_error as mse

mses = pd.Series({'OLS': mse(y, m1.predy.flatten()),
                  'Lag': mse(y, m3.predy_e)
                    })
mses.sort_values()

Lag    23.530225
OLS    23.574139
dtype: float64

In [24]:
GM = ['Manchester', 'Bolton', 'Bury (England)', 'Oldham (England)', 'Rochdale', 'Salford', 'Stockport', 'Tameside', 'Trafford', 'Wigan']

# get the boundaries of the place (add additional buffer around the query)
boundaries = ox.geocode_to_gdf(GM, buffer_dist=0)

# Let's check the boundaries on a map
boundaries.explore()

C:\Users\1\AppData\Local\Temp\ipykernel_19248\1027148540.py:4: UserWarning: The buffer_dist argument as been deprecated and will be removed in a future release. Buffer your results directly, if desired.
  boundaries = ox.geocode_to_gdf(GM, buffer_dist=0)


ImportError: The 'folium', 'matplotlib' and 'mapclassify' packages are required for 'explore()'. You can install them using 'conda install -c conda-forge folium matplotlib mapclassify' or 'pip install folium matplotlib mapclassify'.